In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from training_testing import validate_model, train_regular
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence

c:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\LSTM


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [5]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [6]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 64
raceDB = build_dataset('gru_inputs_new2_pir_simple_NO_SP.npy', hidden_size ,state_filter=["VIC"], margin_type='boosted_sftmin',v6=True)
raceDB.create_new_weights_v2()
#raceDB.adjust_weights({"Dapto":10, "Gunnedah":10})

['speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price']
(1189515, 21)
(355667, 24)
Latest date = 2023-03-27 00:00:00
num_features_per_dog=15


  0%|          | 0/14994 [00:00<?, ?it/s]

c:\ProgramData\Anaconda3\envs\torch2\Lib\site-packages\tqdm\std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


  0%|          | 0/49755 [00:00<?, ?it/s]

c:\ProgramData\Anaconda3\envs\torch2\Lib\site-packages\tqdm\std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:


number of races = 49755, number of unique dogs = 14994
0        (Healesville, 350.0)
1            (Horsham, 410.0)
2        (The Meadows, 525.0)
3        (Warrnambool, 390.0)
4        (The Meadows, 525.0)
                 ...         
49750    (Warrnambool, 390.0)
49751    (Warrnambool, 450.0)
49752    (Warrnambool, 450.0)
49753    (Warrnambool, 450.0)
49754    (Warrnambool, 450.0)
Length: 49755, dtype: object


In [7]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 39923, Test examples 9832


In [8]:
start_date = datetime.datetime.strptime("2019-12-01", "%Y-%m-%d").date()
end_date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()-datetime.timedelta(1)
period = start_date
batches = []
while start_date<end_date:
   
    print(f"{start_date=}")
    period = min(start_date+datetime.timedelta(365), end_date)
    print(f"{period=}")
    batches.append((start_date,period))
    start_date=period
    #batches.append((start_date,end_date))
# batches.append((period,end_date))
print(batches)

start_date=datetime.date(2019, 12, 1)
period=datetime.date(2020, 11, 30)
start_date=datetime.date(2020, 11, 30)
period=datetime.date(2021, 11, 30)
start_date=datetime.date(2021, 11, 30)
period=datetime.date(2022, 7, 31)
[(datetime.date(2019, 12, 1), datetime.date(2020, 11, 30)), (datetime.date(2020, 11, 30), datetime.date(2021, 11, 30)), (datetime.date(2021, 11, 30), datetime.date(2022, 7, 31))]


In [9]:
batch_races_ids = [] # list of race_ids
j = 0
current_batch = []
for i,r in enumerate(raceDB.raceIDs):   
    _,end_date = batches[j]
    race_date = raceDB.racesDict[r].race_date
    if race_date>end_date:
        print(end_date)

        batch_races_ids.append(current_batch)
        current_batch = []
        j += 1
        if j>len(batches)-1:
            break
    else:
        current_batch.append(r)
print(f"Train examples {[len(x) for x in batch_races_ids]}")

2020-11-30
2021-11-30
2022-07-31
Train examples [14170, 15347, 10104]


In [10]:
train_dogs = []
train_dog_input = []
for bi, batch in enumerate(tqdm(batch_races_ids)):
    print(bi)
    print(batch)
    batch_dogs = []
    batch_dog_input = []
    for i in raceDB.dog_ids:
        dog = raceDB.dogsDict[i]
        train = [dog.races[x] for x in dog.races.keys() if x in batch]
        if train:
            batch_dogs.append(dog)
            batch_dog_input.append(train)
    train_dogs.append(batch_dogs)
    train_dog_input.append(batch_dog_input)

  0%|          | 0/3 [00:00<?, ?it/s]

0
['498695480', '500539128', '500541166', '500541223', '500543188', '500543208', '501007014', '501007015', '501007016', '501007017', '501007018', '501007019', '501007020', '501007021', '501007022', '501007023', '501007024', '501031062', '501031063', '501031064', '501031065', '501031066', '501031067', '501031068', '501031069', '501031070', '501031071', '501031072', '501031073', '501031074', '501031075', '501031076', '501031077', '501031078', '501031079', '501031080', '501031081', '501031082', '501031083', '501031084', '501031085', '501309117', '501309118', '501309119', '501309120', '501309121', '501309122', '501309123', '501309124', '501309125', '501309126', '501309127', '501309128', '501343185', '501343186', '501343187', '501343188', '501343189', '501343190', '501343191', '501343192', '501343193', '501343194', '501343195', '501343196', '501394788', '501394789', '501394790', '501394791', '501394792', '501394793', '501394794', '501394795', '501394796', '501394797', '501394798', '50139479

 33%|███▎      | 1/3 [00:58<01:56, 58.28s/it]

1
['614163566', '614163577', '614163724', '614163725', '614163865', '614166029', '614166030', '614166212', '614166324', '614216481', '614216482', '614216483', '614216484', '614216485', '614216486', '614216487', '614216488', '614216489', '614216490', '614216491', '614216492', '614216493', '614216494', '614216495', '614216496', '614216497', '614216498', '614216499', '614216500', '614216501', '614216502', '614216503', '614216504', '614216505', '614216506', '614216507', '614216508', '614216509', '614216510', '614216511', '614216512', '614216513', '614216514', '614216515', '614216516', '614216517', '614216518', '614216519', '614216520', '614216521', '614216522', '614216523', '614216524', '614216525', '614246171', '614246172', '614246173', '614246174', '614246241', '614246272', '614246273', '614246366', '614246481', '614246522', '614300649', '614300650', '614300651', '614300652', '614300653', '614300654', '614300655', '614300656', '614300657', '614300658', '614300659', '614300660', '61430066

 67%|██████▋   | 2/3 [02:00<01:00, 60.64s/it]

2
['731965706', '732290194', '732290195', '732291719', '732291720', '732291721', '732291789', '732291848', '732291878', '732297171', '732297172', '732297173', '732297174', '732297175', '732297176', '732297177', '732297178', '732297179', '732297180', '732297181', '732297182', '732297183', '732297184', '732297185', '732297186', '732297187', '732297188', '732297189', '732297190', '732297191', '732297192', '732329736', '732329737', '732329738', '732329739', '732329740', '732329741', '732329742', '732329743', '732329744', '732329745', '732329746', '732329747', '732329748', '732329749', '732329750', '732329751', '732329752', '732329753', '732329754', '732329755', '732329756', '732329757', '732329758', '732333744', '732333745', '732333746', '732333747', '732333748', '732333749', '732333750', '732333751', '732333752', '732333753', '732333754', '732333755', '732336249', '732336250', '732336251', '732336252', '732336253', '732336254', '732336255', '732336256', '732336257', '732336258', '73233625

100%|██████████| 3/3 [02:44<00:00, 54.71s/it]


In [11]:
batch_races = [[raceDB.racesDict[r] for r in inner] for inner in batch_races_ids]

In [12]:
print(f"Train examples {[len(x) for x in train_dogs]}")
print(f"Train examples {[len(x) for x in train_dog_input]}")
print(f"Train examples {[len(x) for x in batch_races]}")
print(f"Train examples {[len(x) for x in batch_races_ids]}")

Train examples [6959, 7054, 5840]
Train examples [6959, 7054, 5840]
Train examples [14170, 15347, 10104]
Train examples [14170, 15347, 10104]


In [13]:
raceDB.create_dogs_test_split_date()

  0%|          | 0/14994 [00:00<?, ?it/s]

In [14]:
raceDB.attach_races_to_dog_inputv2()

  0%|          | 0/14994 [00:00<?, ?it/s]

In [47]:
test_idx = range(0,len(raceDB.test_dog_ids))
packed_x = [pack_sequence([torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in x]], enforce_sorted=False).to('cuda:0') for x in train_dog_input]
packed_y = pack_sequence([torch.stack(n,0) for n in [[z.full_input.to('cuda:0') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]], enforce_sorted=False)

In [16]:
# train_dog_input_stacked = [[torch.stack(n,0) for n in [[z.full_input.to('cpu') for z in inner] for inner in x]] for x in train_dog_input]

In [17]:
# [sum([len(x) for x in n])/len([len(x) for x in n]) for n in train_dog_input_stacked]

In [48]:
raceDB.batches = {'num_batches':len(train_dogs),'dogs':train_dogs,'train_dog_input':train_dog_input,'batch_races':batch_races,'batch_races_ids':batch_races_ids,'packed_x':packed_x, 'packed_y':packed_y}

In [26]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss", summary="min")
      wandb.define_metric("test_accuracy", summary="max")
      wandb.define_metric("bfprofit", summary="max")
      wandb.define_metric("multibet profit", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy

      model = rnn_classes.GRUNetv2(85,config['hidden_size'], num_layers=config['num_layers'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_450.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        model.load_state_dict(model_data['model_state_dict'])
        config['parent model'] = prev_model_file

      raceDB.to_cuda()

      # criterion = nn.SmoothL1Loss(reduction='mean', beta=5)
      criterion = nn.CrossEntropyLoss(reduction='none')
      # optimizer = optim.AdamW(model.parameters(), lr=0.0001, set_t)
      # optimizer = optim.Adadelta(model.parameters())
      optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])

      # model = torch.compile(model)

      model = model.to(device)

      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_gru.train_regular_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt, Exception) as error:
        print(error)
        print("finished Early")
        
      dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, dataset.hidden_states_dict_gru_v6 , model_name="long nsw new  22000 RUN")
      if sweep:
        raceDB.reset_all_lstm_states
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [20]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':hidden_size,'num_layers':3,'batch_size': 750, 'dropout': 0.3, 'epochs': 10_000, 'f1_layer_size': 256, 'f2_layer_size': 64 , 'learning_rate': 0.0005, 'loss': 'L1', 'l1_beta':0.1,   'optimizer': 'adamW', 'validation_split': 0.1,'batch_before_backwards':7}

In [59]:
#(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static)
importlib.reload(training_testing_gru)
importlib.reload(rnn_classes)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


<module 'rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\rnn_classes.py'>

In [49]:
raceDB.reset_hidden()

In [51]:
torch.cuda.empty_cache()

In [52]:
import wandb

In [60]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

{'hidden_size': 64, 'num_layers': 3, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0005, 'loss': 'L1', 'l1_beta': 0.1, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}
10000
{'hidden_size': 64, 'num_layers': 3, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'f1_layer_size': 256, 'f2_layer_size': 64, 'learning_rate': 0.0005, 'loss': 'L1', 'l1_beta': 0.1, 'optimizer': 'adamW', 'validation_split': 0.1, 'batch_before_backwards': 7}
GRUNetv2(
  (gru): GRU(85, 64, num_layers=3)
  (batchnorm): BatchNorm1d(85, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): ReLU()
  (fc0): Linear(in_features=64, out_features=1, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=582, out_features=64, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=64, out_features=8, bias=True)
  (output_fn): Id

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Exec time of t1 -5.798505200000363
Exec time of t2 3.9101877999928547
Exec time of hidden 5.44084660000226
Exec time of hidden2 15.704600400000345
Exec time of hidden2 0.039615100002265535
Exec time of t_p2 0.009394599997904152
Exec time of t_p3 16.312750199998845
Exec time of t1 -5.465640700000222
Exec time of t2 3.897294900001725
Exec time of hidden 3.5372624000010546


In [ ]:
race = raceDB.get_race_input([1,2])[0]

In [ ]:
race.hidden_in.shape

In [ ]:
torch.rand([1395, 3, 64])

In [ ]:
x = torch.rand([1395, 3, 64])
x[100]

In [ ]:
x.transpose(0,1)

In [ ]:
torch.zeros(3,5,10)

In [ ]:
len_test = len(raceDB.test_dog_ids)
test_idx = range(0,len_test)

# dog_inputs = [[z.full_input for z in inner] for inner in [x for x in raceDB.train_dogs.values()]]
# Uses train dogs here because only dogs in Train set will have new hidden values?? Does that make sense?
# no will jumble around but wont throw error in feeding model, because when building dogs sorted, 
# lengths will be made to match size of X(test)
# Shouldn't have much of an effect due to size of test set??
dogs = [x for x in  raceDB.test_dogs.values()]  #[Dog]
dog_input = [inner for inner in [x for x in raceDB.get_dog_test(test_idx)]] #[[DogInput]]

# for dog in dogs:
#     dog.hidden = dog.hidden.to('cuda:0')

train = [torch.stack(n,0) for n in [[z.full_input.to('cpu') for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]]
X = pack_sequence(train, enforce_sorted=False).to('cuda:0')

dogs_sorted = [dogs[x] for x in X[3]]

In [ ]:
hidden_in = torch.stack([x.hidden for x in dogs])

In [ ]:
hidden_in.shape

In [ ]:
hidden_in.reshape((3, 1439, 64))

In [ ]:
dogs = [x for x in  raceDB.test_dogs.values()][:5]

In [ ]:
[d.hidden for d in dogs]

In [ ]:
set(dog.races.keys()).intersection(set(raceDB.test_race_ids))
set(dog.races.keys()).intersection(set(raceDB.test_race_ids))

In [ ]:
[r.race.raceid for r in dog.test]

In [ ]:
for i in set(dog.races.keys()).intersection(set(raceDB.test_race_ids)):
    print(i)

In [ ]:
[dog.races[x] for x in set(dog.races.keys()).intersection(set(raceDB.test_race_ids))]

In [ ]:
dog.test

In [ ]:
dog_input = [inner for inner in [x for x in raceDB.get_dog_train([0,1,3,4,5,6])]][2]

In [ ]:
races = [r.race for r in dog_input]

In [ ]:
pd.Series([r.race_date for r in races])

In [ ]:
raceDB.create_hidden_states_dict_v2()

In [ ]:
model_saver(model,optimizer,450,0.2,raceDB.hidden_states_dict_gru_v6, model_name="long run to use")

In [ ]:
raceDB.hidden_states_dict_gru_v6

In [ ]:
criterion = nn.CrossEntropyLoss(reduction='none')

In [ ]:
training_testing_gru.validate_model_v2(model,raceDB,criterion)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
len_test = len(raceDB.test_dog_ids)
test_idx = range(0,len_test)

# dog_inputs = [[z.full_input for z in inner] for inner in [x for x in raceDB.train_dogs.values()]]
dogs = [x for x in  raceDB.test_dogs.values()]  #[Dog]
dog_input = [inner for inner in [x for x in raceDB.get_dog_test(test_idx)]] #[[DogInput]]

for dog in dogs:
    dog.hidden = dog.hidden.to('cuda:0')

train = [torch.stack(n,0) for n in [[z.full_input for z in inner] for inner in [x for x in raceDB.get_dog_test(test_idx)]]]
X = pack_sequence(train, enforce_sorted=False)

dogs_sorted = [dogs[x] for x in X[3]]
hidden_in = torch.unsqueeze(torch.stack([x.hidden for x in dogs_sorted]).to('cuda:0'),0)

In [ ]:
batchnorm = torch.nn.BatchNorm1d(196)

In [ ]:
batchnorm(X)

In [ ]:
loss = nn.CrossEntropyLoss(reduction='none')
X = torch.rand(5,8)
Y = torch.rand(5,8)
loss(X,Y)

In [ ]:
for i,dog in enumerate(dog_input):
    dog_outputs = output[i]
    for j,di in enumerate(dog):
        di.output = dog_outputs[j]

In [ ]:
raceDB.margin_from_dog_to_race()

In [ ]:
torch.tensor(100.1).to('cuda:0')

In [ ]:
torch.tensor([100.1]).to('cuda:0')

In [ ]:
for r in raceDB.test_race_ids:
    race = raceDB.racesDict[r]
    outputs = []
    for d in race.dogs:
        print(d)
        print(d.output)
        outputs.append(d.output)
    break
    print(output)
    race.margins = torch.stack(outputs)

In [ ]:
torch.cat(outputs)

In [ ]:
output[0]

In [ ]:
test_races = [raceDB.racesDict[x] for x in raceDB.test_race_ids][0]

In [ ]:
test

In [ ]:
print(test_races.margins)

In [ ]:
[x.dog.dog_name for x in test_races.dogs]

In [ ]:
sweep_config = {"method": "random"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "f1_layer_size": {"values": [256]},
    "f2_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "hidden_size": {"value":64}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [100]},
        "validation_split": {"value": 0.1},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        "num_layers": {"values": [2]},
    }
)

parameters_dict.update(
    {
        "learning_rate": {
            # a flat distribution between 0 and 0.1
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.001,
        },
        "l1_beta": {"value": 0.1
        },
        "batch_size": {
            'values': [50,125,250,500]
        },
        "batch_before_backwards": {
            'values': [5,10,15,50]
        }
    }
)

import pprint

pprint.pprint(sweep_config)


sweep_config

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="LSTM_sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

In [ ]:

stop
model = GRUNet(203,64).to(device)
model.load_state_dict(model_state_dict)

In [ ]:

dataset.create_hidden_states_dict()